# Complex datasets

Now lets work with a much larger and more realistic dataset, just to get a feel for what working with big data is like. This will help you get the hang of working with datasets that share multiple coordinates and that have high complexity.

We are going to be using current oceanic wave modelling projections from NOAA, using their sophisticated WAVEWATCH III® (Tolman 1997, 1999a, 2009) model. See [the central model page](https://polar.ncep.noaa.gov/waves/wavewatch/) for more info.

This will also allow us to demonstrate the use of OpenDAP access to big datasets over http, which is an awesome feature for working with big data that allows lazy dataset loading.

In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the folder
# which was created in your username directory under /scratch/vp91

#TODO 
os.chdir(os.path.expandvars("/scratch/vp91/$USER/Data-Analytics/"))


In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt 
import matplotlib.animation as animation 
# plotting
import hvplot.xarray
import hvplot.pandas
import cartopy.crs as ccrs
import geoviews

# note adapted from pangeo example gallery 

Using `xarray.open_mfdataset()` function we can combine multiple netCDF files as one Xarray dataset object. Here we use `*` in `datapath` for pattern matching to select all files those names ending with `.nc`.

In [ ]:
data_path = "/scratch/vp91/AAPP2023/Data/sea_surf_temp_data/*.nc"

Lets open our dataset!

In [ ]:
ds = xr.open_mfdataset(data_path)
ds

## Breaking down our dataset

Okay wow that looks complicated. Lets break it down one by one. 

Firstly it's important to note that our dataset will be lazily loaded over the network when we index it to save transferring GBs worth of data in one hit. See the [xarray page on input/output](https://docs.xarray.dev/en/stable/user-guide/io.html) for more details.

First lets look at how big it is and our data variables:

In [ ]:
ds.nbytes/1e9 #GBs

In [ ]:
ds.data_vars

We have 1 variable in our dataset. This table really highlights the strenghts of xarray, that is working with **labelled multidimensional data**. 

Also examine the **metadata** of this dataset by looking at the `attributes`.

Note in the right hand coloumn what `coords` the data corresponds to. For example an entry like:

```
sst                                           (time1, lat, lon) float32 ...

```

indicates that the `sst` variable exists on the `time1, lat and lon ` coordinates. In other words it is time dependent 3D data on the surface of the earth as we might expect. 

Lets have a closer look at this variable

In [ ]:
var = 'sst'
ds[var]

As we might expect it is our familiar `DataArray`! It has the expected dimensions and a whole bunch of metadata. Lets investigate our dataset coordinates:

In [ ]:
ds.coords

We have latitude, longitude, and a time variable.

----------------------------------------------------------

Now that we known whats happening with our dataset lets make some plots! We are going to use `hvplot` a high level plotting utility based on `Bokeh` that supports the kind of global map we want.

We will plot in some of the data by slicing the array. Note that the interactive plot is best played with once an other computation is done.

In [ ]:
ds[var][:,:,:].hvplot(x='lon', y='lat', cmap='rainbow')

Wow that was simple! This shows the power of  `xarray` and its associated stacks that enable easy manipulation of complex data. We can also use `cartopy` to get a global orthographic projection that we can center on Australia.

In [ ]:
crs = ccrs.Orthographic(central_longitude=120, central_latitude=-30)


In [ ]:
ds[var][0:-1:1000,:,:].hvplot(x='lon', y='lat',
                            cmap='rainbow', coastline=True, geo=True,
                            project=True, projection=crs, rasterize=True,
                            widget_type='scrubber', widget_location='bottom')

Note that the time scrubber doesn't work for the above plot due to some hvplot peculiarities, but you can make a movie  by saving the images and then gluing them together.

Now that we have explored our dataset, lets do a few operations.

Due to some pecularities of WaveWatch III that I don't quite understand, the time dimension can sometimes be `time` or sometimes `time1`. Lets figure out which one it is. 

In [ ]:
tvar = [var for var in ds[var].dims if "time" in var]
tvar = tvar[0]
tvar

this is the timespan that our dataset runs over, its in 3 hour increments.

In [ ]:
ds.sst.nbytes/1e9 #GBs

Lets take a latitudinal slice and show the variation in sst over time and longitude.

In [ ]:
ds.sst.sel(lat=82.0).plot(x="lon", y=tvar, cmap="viridis")

## Grouping

One of xarray's very powerful tools tools is the ability to use the "split-apply-combine" paradigm which should be familiar to users of pandas. 

Here we group by day by using the `.day` attribute of the `datetime64` format.

In [ ]:
gb = ds["sst"].groupby(tvar +'.year')
gb

Awesome! We got a gropuby object back that allows us to do operations over the grouped dataset.

Lets go ahead and do that by computing a mean across the last 10 years. We do this with the usual syntax on the `GroupBy` object. 

Note that this computation is **quite expensive** and can take a while. If you are taking this course in tandem with the  `Dask` component, we will explore ways to make this computation faster later on.

In [ ]:
gb.mean(dim=tvar)[-11:,:,:].hvplot(x='lon', y='lat', cmap='rainbow')

Neat, we now have a daily average for our `Wind_speed_surface` variable.  Hopefully the interactive plot should be working for everyone.

We are not going to cover much more on computations on this large dataset as they can be a bit slow. The sky is the limit however and `xarray` comes with lots of built in super useful stuff for us, such as methods for computing rolling averages using `.rolling()` and the capacity to do almost anything you can do in numpy and/or pandas.

## Working with time, resampling and interpreting

Sometimes the timesteps we want to examine are larger or smaller than the data we have. We can use `resample` to downsample or upsample our data and `interp` to estimate the value at new timepoints that were not observed.

First lets downsample into 3 months intervals.

This is different from grouping as we have the flexibility to aggregate over different timeperiods. However we cannot use resample to group by values in categorical columns as you can do with `groupby`. I like to think of resample as a row wise aggregation only. 

In [ ]:
ds_downsample = ds.sst.resample(time='3m').mean(tvar)
ds_downsample.hvplot(x='lon', y='lat', cmap='rainbow')

We can use resample to upsample  as well. Here we do it over 7 days intervals, slightly more then the 1 month intervals in the model using a linear interpolation. Note that we didn't have to specify an aggregator, but instead an interpolation.

In [ ]:
ds_upsample = ds.sst.resample(time='7d').interpolate("linear")
ds_upsample.hvplot(x='lon', y='lat', cmap='rainbow')

### Challenge

Perform an upsample to an even finer resolution (not too fine) and use a quadratic interpolation. You can also make a plot

In [ ]:
# upsample

<details><summary><b>Solution</b></summary>
   <pre>
    <br> ds_upsample = ds.sst.resample(time='12h').interpolate("qudratic")
ds_upsample.hvplot(x='lon', y='lat', cmap='rainbow')
   </pre>
</details>

Down and upsampling is super powerfull and was again super easy all things considered.

## Conclusion

You have now worked with big data and complex `xarray` datasets in a large oceanographic model.  You are hopefully now familiar with all the basic concepts of `xarray`. Of course there are more details in the manual, but hopefully this is enough to get you started. 
